In [1]:
%load_ext Cython
# https://kaushikghose.wordpress.com/2014/12/08/get-more-out-of-cython/
# http://nealhughes.net/cython1/
# http://codereview.stackexchange.com/questions/43413/optimize-cython-code-with-np-ndarray-contained
# https://github.com/bendmorris/pybioclim/blob/master/src/coords.pyx

In [2]:
import math
import numpy as np
import pandas as pd
import logging
from datetime import datetime, timezone
from datetime import timedelta
import os
import pytz


GEOLIFE_DATE_FORMAT = "%Y-%m-%d %H:%M:%S"
GMT = pytz.timezone('GMT')
BEIJING_TIMEZONE = pytz.timezone('Asia/Shanghai')


gpsdata = np.array([[53.3854313241815,-1.4932127483203013, 1384209632918], 
                    [53.38609362252086,-1.4951305729482556, 1384209650218]])


def calc_distance(lat1, lon1, lat2, lon2, unit="km"):
    """
    The function returns the distance between two lat, long coordinates in miles or kilometers

    :param lat1: latitude info, in geo reference WGS84
    :param lon1: longitude info, in geo reference WGS84
    :type arg1: float
    :type arg2: float
    :type arg3: float
    :type arg4: float
    :return: the distance between two coordinates in mile or km
    :rtype: float

    :Example:
    >>> calc_distance(40.920320, -74.293288, 40.730975, -74.001509, "km")
    """
    error_msg = "The unit for distance has to be one of mile and km"
    if unit not in ['mile', 'km']:
        raise ValueError(error_msg)
    theta = lon1 -lon2
    dist_angle = math.sin(math.radians(lat1))*math.sin(math.radians(lat2)) \
            + math.cos(math.radians(lat1))*math.cos(math.radians(lat2)) \
            * math.cos(math.radians(theta))
    dist_angle = max(-1, min(dist_angle, 1))
    dist_angle = math.acos(dist_angle)
    dist_angle = math.degrees(dist_angle)
    mile = dist_angle * 60.0 * 1.1515
    km = mile * 1.609344
    if unit == 'mile':
        return mile
    elif unit == 'km':
        return km
    else:
        raise ValueError(error_msg)


def load_gps_plt_data2array(file_dir):
    """
    Load in the gps data in the log file if file directory is given
    Field 1: Latitude in decimal degrees.
    Field 2: Longitude in decimal degrees.
    Field 3: All set to 0 for this dataset.
    Field 4: Altitude in feet (-777 if not valid).
    Field 5: Date - number of days (with fractional part) that have passed since 12/30/1899.
    Field 6: Date as a string.
    Field 7: Time as a string.
    >>> x_ = load_gps_plt_data2array(file_dir='./data_GeoLife/001/Trajectory/20081023055305.plt')
    """
    gpsdata = np.genfromtxt(file_dir,
                            delimiter=',',
                            skip_header=6,
                            dtype="f8,f8,f8,f8,f8,S10,S8",
                            #dtype=('<f8','<f8','<f8','<f8','<f8','|S10', '|S8'),
                            #dtype=(float,float,float,float,float,'|S10','|S10'),
                            usecols = range(7), # select your own columns
                            names=['lat', 'lon', 'dummy1', 'alt', 'days_passed', 'date_str', 'time'],
                            autostrip=True)
    return gpsdata


def load_gps_plt_data(file_dir):
    """
    Load in the gps data in the log file if file directory is given
    Field 1: Latitude in decimal degrees.
    Field 2: Longitude in decimal degrees.
    Field 3: All set to 0 for this dataset.
    Field 4: Altitude in feet (-777 if not valid).
    Field 5: Date - number of days (with fractional part) that have passed since 12/30/1899.
    Field 6: Date as a string.
    Field 7: Time as a string.
    >>> x_ = load_gps_plt_data(file_dir='./data_GeoLife/001/Trajectory/20081023055305.plt')
    """
    # gpsdata = pd.read_csv(file_dir, sep=',', header=None, skiprows=6)  # do not use any row for column names by header=None
    gpsdata = pd.read_csv(file_dir, sep=',', header=None, skiprows=6, names=['lat', 'lon', 'dummy1', 'alt', 'days_passed', 'date_str', 'time'])
    gpsdata['datetime'] = gpsdata.ix[:, 5] + ' ' + gpsdata.ix[:, 6]
    # to convert time to Beijing time
    # gpsdata['datetime'] = pd.DatetimeIndex(pd.to_datetime(gpsdata['datetime'],format='%Y-%m-%d %H:%M:%S'),tz='UTC').tz_convert('Asia/Shanghai')
    gpsdata['datetime'] = pd.DatetimeIndex(pd.to_datetime(gpsdata['datetime'],format='%Y-%m-%d %H:%M:%S'),tz='UTC')
    # sample conversion from unix epoch to datetime
    # np.array([1368431149, 1368431150]).astype('datetime64[s]')
    # convert to UTC epoch time in million seconds, first conversion below converts to nano second
    gpsdata['timestamp'] = gpsdata['datetime'].astype(np.int64)/(10**6)
    gpsdata['timestamp'] = gpsdata['timestamp'].astype(np.int64)
    # remove the records with missing date time
    gpsdata = gpsdata[gpsdata['timestamp'] > 0]
    return gpsdata


In [3]:
print(calc_distance(40.920320, -74.293288, 40.730975, -74.001509, "km"))
gps = load_gps_plt_data(file_dir='./data_GeoLife/001/Trajectory/20081023055305.plt')
gps.ix[:10, [0,1,8]]

32.340596225930696


,lat,lon,timestamp
0,39.984094,116.319236,1224741185000
2,39.984224,116.319402,1224741191000
3,39.984211,116.319389,1224741196000
4,39.984217,116.319422,1224741201000
5,39.984710,116.319865,1224741203000
6,39.984674,116.319810,1224741208000
7,39.984623,116.319773,1224741213000
8,39.984606,116.319732,1224741218000
9,39.984555,116.319728,1224741223000
10,39.984579,116.319769,1224741228000


In [4]:
distance = sum([calc_distance(gpsdata[i][0], 
                              gpsdata[i][1], 
                              gpsdata[i+1][0], 
                              gpsdata[i+1][1], "km") 
                for i in range(len(gpsdata)-1)])
print("Distance traveled on GPS is {} km".format(distance))

######################


Distance traveled on GPS is 0.14696399289179068 km


In [5]:
%%cython
# or use %%cython -a for more details

cimport cython

cdef extern from "math.h":
    double sin(double)
    double cos(double)
    double acos(double)
    double atan2(double, double)

cdef double pi = 3.141592654

@cython.cdivision(True)
cdef double radians(double deg):
    return deg/180.*(pi)

@cython.cdivision(True)
cdef double degrees(double rad):
    return rad * (180.0 / pi)

# from libc.math cimport acos, degrees, sin, cos, radians

cdef double calc_distance_km_c(double lat1, double lon1, 
                    double lat2, double lon2):
    cdef double theta, dist_angle, mile, km
        
    theta = lon1 -lon2
    dist_angle = sin(radians(lat1))* sin(radians(lat2)) \
            + cos(radians(lat1))* cos(radians(lat2)) \
            * cos(radians(theta))
    dist_angle = max(-1, min(dist_angle, 1))
    dist_angle = acos(dist_angle)
    dist_angle = degrees(dist_angle)
    mile = dist_angle * 60.0 * 1.1515
    km = mile * 1.609344
    return km

print(calc_distance_km_c(40.920320, -74.293288, 40.730975, -74.001509))

32.34059622449445


In [6]:
%%cython

cimport numpy as np
cimport cython
from libc.time cimport difftime
import calendar, time


cdef extern from "math.h":
    double sin(double)
    double cos(double)
    double acos(double)
    double atan2(double, double)
    double log(double)

cdef double pi = 3.141592654

@cython.cdivision(True)
cdef double radians(double deg):
    return deg/180.*(pi)

@cython.cdivision(True)
cdef double degrees(double rad):
    return rad * (180.0 / pi)

# from libc.math cimport acos, degrees, sin, cos, radians

cdef double calc_distance_km_c(double lat1, double lon1, 
                    double lat2, double lon2):
    cdef double theta, dist_angle, mile, km
        
    theta = lon1 -lon2
    dist_angle = sin(radians(lat1))* sin(radians(lat2)) \
            + cos(radians(lat1))* cos(radians(lat2)) \
            * cos(radians(theta))
    dist_angle = max(-1, min(dist_angle, 1))
    dist_angle = acos(dist_angle)
    dist_angle = degrees(dist_angle)
    mile = dist_angle * 60.0 * 1.1515
    km = mile * 1.609344
    return km

#cdef np.ndarray h = np.zeros([xmax, ymax], dtype=DTYPE)

def speed_cal_c(np.ndarray[np.float_t, ndim=2] gpsdata):
    """
    :param gpsdata: numpy 2 dimensional gps data, lat, long and UTC timestamp
    :return Speed in the gps data, km/hour
    """
    cdef int len_ = len(gpsdata)-1
    cdef double distance = 0.
    cdef double scale2sec, x_, x, run_time_in_sec, speed
    for i in range(len_):
        distance += calc_distance_km_c(gpsdata[i][0], gpsdata[i][1], gpsdata[i+1][0], gpsdata[i+1][1])
    # x_ = calendar.timegm(time.gmtime())
    x_ = 1187142632
    scale_ = gpsdata[0, 2] / x_
    scale_ = min([1.0, 10.0, 100.0, 1000.0], key=lambda x:abs(log(x)-log(scale_)))
    run_time_in_sec = max(gpsdata[:, 2])/scale_ - min(gpsdata[:, 2])/scale_
    speed = (distance / (run_time_in_sec / 3600.0))
    print("The distance traveled is {} km".format(distance))
    print("The time traveled is {} seconds".format(run_time_in_sec))
    print("The returned speed is in unit km/h")
    return speed


In [7]:
speed_cal_c(gpsdata)

The distance traveled is 0.14696396221255864 km
The time traveled is 17.299999952316284 seconds
The returned speed is in unit km/h


30.582096267253124